In [0]:
!zip -r /content/file.zip /content/logss_model_dense

  adding: content/logss_model_dense/ (stored 0%)
  adding: content/logss_model_dense/events.out.tfevents.1570487317.dea8a2611466 (deflated 92%)
  adding: content/logss_model_dense/plugins/ (stored 0%)
  adding: content/logss_model_dense/plugins/profile/ (stored 0%)
  adding: content/logss_model_dense/plugins/profile/2019-10-07_22-28-53/ (stored 0%)
  adding: content/logss_model_dense/plugins/profile/2019-10-07_22-28-53/local.trace (deflated 93%)
  adding: content/logss_model_dense/events.out.tfevents.1570487333.dea8a2611466.profile-empty (deflated 5%)


In [0]:
from google.colab import files
files.download("/content/file.zip")

In [0]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau 

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
# from tensorflow import keras

# from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
# import tensorflow as tf
# tf.config.gpu.set_per_process_memory_fraction(0.75)
# tf.config.gpu.set_per_process_memory_growth(True)
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

# Create a session with the above options specified.
# k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Load CIFAR10 Data
num_classes = 10
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

In [7]:
X_train.shape

(50000, 32, 32, 3)

In [8]:
X_test.shape

(10000, 32, 32, 3)

In [0]:
# Dense Block
compression = 0.5
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    Conv1 = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(AvgPooling)
    Conv2 = layers.Conv2D(num_classes, (1,1) ,padding='valid')(Conv1)
    Global_pool = layers.GlobalAveragePooling2D()(Conv2)
    
    output = layers.Activation('softmax')(Global_pool)
    return output

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 35
l = 6
num_filter = 64
compression = 0.5
dropout_rate = 0

In [11]:
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
#https://arxiv.org/pdf/1608.06993.pdf
# from IPython.display import IFrame, YouTubeVideo
# YouTubeVideo(id='-W6y8xnd--U', width=600)

In [13]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 64)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

mean = X_train.mean(0)
std = X_train.std(0)
def preprocess_data(data_set):
    # mean = np.array([125.3, 123.0, 113.9])
    # std = np.array([63.0, 62.1, 66.7])

    data_set -= mean
    data_set /= std
    return data_set

X_train = preprocess_data(X_train)
X_test = preprocess_data(X_test)

In [16]:
# Data augementation
from keras.preprocessing.image import ImageDataGenerator
datagen_train = ImageDataGenerator(
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True,
)

datagen_train.fit(X_train)

Using TensorFlow backend.


In [0]:
checkpoint_3 = ModelCheckpoint("model_dense.h5",monitor="val_acc",mode="max",save_best_only = True,verbose=1) 
NAME = 'model_dense' 
tensorboard_2 = TensorBoard(log_dir='logss\{}'.format(NAME),update_freq='epoch',batch_size=batch_size) 
callbacks_2 = [tensorboard_2,checkpoint_3]


In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [22]:
history = model.fit_generator(datagen_train.flow(X_train, y_train, batch_size=batch_size),steps_per_epoch=(len(X_train)/batch_size)*5,
    epochs=epochs,
    verbose = 1,
    validation_data=(X_test, y_test),
    callbacks = callbacks_2
)

Epoch 1/35
10000/1953 [=========================================================================================================================================================] - 10s 968us/sample - loss: 0.7712 - acc: 0.7534

Epoch 00001: val_acc improved from -inf to 0.75340, saving model to model_dense.h5
1954/1953 [==============================] - 829s 424ms/step - loss: 0.8581 - acc: 0.6952 - val_loss: 0.7602 - val_acc: 0.7534
Epoch 2/35
10000/1953 [=========================================================================================================================================================] - 9s 855us/sample - loss: 0.5275 - acc: 0.7822

Epoch 00002: val_acc improved from 0.75340 to 0.78220, saving model to model_dense.h5
1954/1953 [==============================] - 813s 416ms/step - loss: 0.4530 - acc: 0.8430 - val_loss: 0.6827 - val_acc: 0.7822
Epoch 3/35
10000/1953 [=====================================================================================================

In [25]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 11s 1ms/sample - loss: 0.4859 - acc: 0.9141
Test loss: 0.4859287844727747
Test accuracy: 0.9141


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk
